In [1]:
#import block
# some of these may not be needed
import numpy as np
from astropy.io import fits
import pdb
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import rc
import matplotlib
from astropy.visualization import hist
from datetime import datetime
from pathlib import Path
import os
from ccdproc import ImageFileCollection
import ccdproc as ccdp
from astropy.modeling.models import Polynomial1D,Chebyshev1D,Legendre1D,Hermite1D, Gaussian1D,Gaussian2D,Polynomial2D
from astropy.modeling import fitting
from astropy.stats import mad_std
from astropy.nddata import CCDData, StdDevUncertainty
from scipy import stats
from astropy import units as u
from photutils.aperture import CircularAperture,CircularAnnulus
from photutils.aperture import aperture_photometry
from astropy.stats import sigma_clipped_stats
from photutils.detection import DAOStarFinder
from astropy.table import Table
from imexam.imexamine import Imexamine
from astropy.coordinates import SkyCoord

Ginga not installed, use other viewer, or no viewer


In [2]:
# grab show_image
phot_tutorial_dir = '/Users/Kira Simpson/Desktop/ASTR8060/repositories/notebooks'
import sys
sys.path.insert(0,phot_tutorial_dir)
from convenience_functions import show_image

# plotting defaults
# Use custom style for larger fonts and figures
plt.style.use(phot_tutorial_dir+'/guide.mplstyle')

# Set some default parameters for the plots below
rc('font', size=12)
rc('axes', grid=True)

In [3]:
# define directories
data_dir = '/Users/Kira Simpson/Desktop/ASTR8060/data/Imaging/'
reduced_dir = '/Users/Kira Simpson/Desktop/ASTR8060/data/im_reduc/' #working directory

# 1.) 

In [37]:
#use the get_phot function from Prof. Runnoe's solution for HW6.
#this function will be cannibalized later for NGC6823, but this version is useful for PG1633+099 instrumental magnitudes.

def get_phot(file,approx_pos,filtr):
    '''
    This function takes an input file, list of positions for stars to find, and plot title.
    
    It executes the following steps:
    1. Read in the data.
    2. Find all sources in the image. It assumes the FWHM is approximately 4 pix and finds everything greater than 10 sigma.
    3. It collects the info for the sources given in the input position list, matching within 10 pixel positions.
    4. It puts r=10pix apertures on each of these stars, as well as r_in=15, r_out=20 background annuli.
    5. It calculates background-subtracted instrumental magnitudes.
    6. It plots the image and apertures for visual inspection.
    7. It returns a table of photometry.
    '''
    fwhm       = 4.0
    source_snr = 10

    sci        = CCDData.read(file,unit='adu')
    data       = sci.data
    hdr        = sci.header
    
    mean, median, std = sigma_clipped_stats(data, sigma=3.0)    
    daofind = DAOStarFinder(fwhm=fwhm, threshold=source_snr*std)    
    sources = daofind(data)
    
    positions = np.zeros_like(approx_pos)
    for position,i in zip(approx_pos,range(np.shape(approx_pos)[0])):
        match = np.where((abs(sources['xcentroid']-position[0])<10) & (abs(sources['ycentroid']-position[1])<10))[0][0]
        positions[i] = (sources['xcentroid'][match],sources['ycentroid'][match])
          
    # photometry
    # biggest FWHM is 4.2 pix
    aperture  = CircularAperture(positions, r=4)
    phot = aperture_photometry(data, aperture)
    for col in phot.colnames:
        phot[col].info.format = '%.8g'              # for consistent table output
        
    # calculate the mean background
    # note that for PG1633+099, this will include two little stars :(
    annulus_aperture = CircularAnnulus(positions, r_in=15., r_out=20.)
    bg   = aperture_photometry(data,annulus_aperture)
    for col in bg.colnames:
        bg[col].info.format = '%.8g'                # for consistent table output
    msky = bg['aperture_sum']/annulus_aperture.area # get bg cts/pixel
    
    phot['inst_mag'] = -2.5 * np.log10(phot.columns['aperture_sum']-msky*aperture.area)
    phot['diff_mag'] =phot.columns['inst_mag']-lan_mag
    phot['V']   = [14.397,15.256,12.969,13.229,13.691]
    phot['B-V'] = [-0.192,0.837,1.081,1.134,0.535]

    
    # show the image with apertures
    #fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    #show_image(data, cmap='gray', ax=ax, fig=fig)
    #aperture.plot(color='green', lw=1.5, alpha=0.5) # apertures
    #annulus_aperture.plot(color='red', lw=1.5, alpha=0.5)                        # bg annulus
    #ax.set_xlim([800,1500])
    #ax.set_ylim([800,1500])
    #plt.title(title)
    #plt.tight_layout()
    #plt.show()
    
    return phot

In [38]:
#taken from Prof. Runnoe's solutions as well.

approx_pos = [[(988.8683,1049.2803),(1044.8029,1043.2159),(1251.6695,1221.4295),(1363.9839,1230.9691),(1444.0144,1182.0372)],\
[(987.9844,1049.2036),(1043.8968,1043.1361),(1250.6889,1221.1958),(1363.0112,1230.7516),(1443.0305,1181.8385)],\
[(931.8118,1043.4529),(987.6939,1037.3564),(1194.6650,1215.5002),(1307.0202,1225.0282),(1386.9956,1176.1044)],\
[(932.0204,1043.5176),(987.9188,1037.3954),(1194.7984,1215.5333),(1307.1397,1225.0323),(1387.0961,1176.0627)],\
[(924.4201,1037.0850),(980.2808,1030.9915),(1187.3280,1208.8491),(1299.5213,1218.2871),(1379.4235,1169.3869)],\
[(925.3480,1036.7100),(981.2349,1030.6284),(1188.1673,1208.4725),(1300.4663,1217.9357),(1380.4332,1168.9797)],\
[(838.3982,1039.8684),(894.1732,1033.8111),(1101.0422,1211.7489),(1213.2663,1221.1237),(1293.3098,1172.1284)],\
[(838.1403,1039.7337),(893.9993,1033.6562),(1100.9368,1211.3933),(1213.1794,1220.8576),(1293.1324,1171.9191)]]

#landolt standard magnitudes for UBVRI
lan_mag = [6,6,6,6,5]
lan_mag2 = [6,6,6,6,5,6,6,6,6,5]

In [ ]:
#from Prof. Runnoe's HW6 solution
imgs       = ccdp.ImageFileCollection(reduced_dir,glob_include='*otzf.fits')
sci_files  = imgs.files_filtered(imagetyp='science',object='pg1633',include_path=True)
sci = ((imgs.summary['imagetyp'] == 'science') & (imgs.summary['filter'] != 'Ha'))
sci_filters = set(imgs.summary['filter'][sci])


standards = Table()
standards['filter'] = [filtr for filtr in sci_filters]
standards['k'] = np.zeros(len(sci_filters))

airmass = [1.18,1.24,1.51,2.21]


for filtr,ii in zip(sci_filters,range(len(sci_filters))):
    # get files in this filter
    sci_infilter = imgs.files_filtered(imagetyp='science', object='pg1633',filter=filtr,include_path=True)
    
    if filtr=='V': # skip the linearity test
        sci_infilter = sci_infilter[7:]
    
    # do aperture photometry
    phot = [get_phot(file,approx_pos[i],filtr+', '+file.split('/')[-1]) for file,i in zip(sci_infilter,range(len(sci_infilter)))]

    i_diff_mags = [] #i as in instrument
    for i in range(8):
        for j in range(5):
            i_diff_mags.append(phot[i][j][5])
    
    
    plt.figure(9)
    plt.scatter([1.18]*10,i_diff_mags[:10])
    plt.scatter([1.24]*10,i_diff_mags[10:20])
    plt.scatter([1.51]*10,i_diff_mags[20:30])
    plt.scatter([2.21]*10,i_diff_mags[30:40])
    plt.title('Instrumental Magnitudes vs. Airmass')
    plt.show()

In [ ]:
#make a list of airmasses
airmass = [1.18]*10+[1.24]*10+[1.51]*10+[2.21]*10
#use a linear regression to find the extinction coefficients for each filter
linregress(np.array(airmass),np.array(i_diff_mags))

# 2.)

What is the flux/magnitude at the top of the atmosphere? (so at an airmass of zero, assuming no time variation and airmass is linear with extinction)

F = F_0 * e^(-tau_0 * chi)

The magnitude version of this equation is:
m_0 = m - 1.085 * tau_0 * chi = m - (k * chi)

This is the difference between observed magnitude and known magnitude (the standard stars from landolt)

for the m - m_0 vs airmass plot, the slope is the extinction coefficient k
 
The following steps will find all the pieces of this equation and apply this correction to all of our magnitudes.

In [ ]:
k = 0.6289580027687809 #pulled from linear regression above

for filtr,ii in zip(sci_filters,range(len(sci_filters))):
    # get files in this filter
    sci_infilter = imgs.files_filtered(imagetyp='science', object='pg1633',filter=filtr,include_path=True)
    
    if filtr=='V': # skip the linearity test
        sci_infilter = sci_infilter[7:]
        j_diff_mags = []
        for i in range(8):
            for j in range(5):
                j_diff_mags.append(phot[i][j][4])

j_diff_mags = np.array(j_diff_mags)
                
m_o1 = j_diff_mags[:10] - k * 1.18

m_o2 = j_diff_mags[10:20] - k * 1.24

m_o3 = j_diff_mags[20:30] - k * 1.51

m_o4 = j_diff_mags[30:40] - k * 2.21

In [ ]:
#corrected magnitudes: observed - standard mags
m_cor = m_o1 - lan_mag2
m_cor2 = m_o2 - lan_mag2
m_cor3 = m_o3 - lan_mag2
m_cor4 =m_o4 - lan_mag2

In [ ]:
#list of the B-V colors pulled from the Landolt paper
BV = [-0.192,0.837,1.081,1.134, 0.535,-0.192,0.837,1.081,1.134, 0.535,-0.192,0.837,1.081,1.134, 0.535,-0.192,0.837,1.081,1.134, 0.535,-0.192,0.837,1.081,1.134, 0.535,-0.192,0.837,1.081,1.134, 0.535,-0.192,0.837,1.081,1.134, 0.535,-0.192,0.837,1.081,1.134, 0.535]

In [ ]:
#collect all the corrected magnitudes
all_mcorr = list(m_cor) + list(m_cor2) + list(m_cor3) + list(m_cor4)

plt.scatter(BV,all_mcorr)
plt.title('Color vs. Corrected Magnitudes')
plt.show()
linregress(BV,all_mcorr)

the linear regression gives us a line equation y = -20.32 + 0.02x

# 3.)

In [ ]:
#apply what we've been doing to an open cluster and see how well this works for us.
nimgs       = ccdp.ImageFileCollection(reduced_dir,glob_include='*otzf.fits')

airmass = [1.18,1.24,1.51,2.21]
#print(airmass)

n_infilter = nimgs.files_filtered(imagetyp='science', object='ngc6823',filter='V',include_path=True)

file = n_infilter[0]
fwhm       = 5.0
source_snr = 20

sci        = CCDData.read(file,unit='adu')
#use a smaller set of the data 
data       = sci.data[739:1438,880:1660]
hdr        = sci.header

mean, median, std = sigma_clipped_stats(data, sigma=3.0)    
daofind = DAOStarFinder(fwhm=fwhm, threshold=source_snr*std)    
sources = daofind(data)

im1_sf_positions = []

for i in range(len(sources['xcentroid'])):
    position = tuple([sources['xcentroid'][i],sources['ycentroid'][i]])
    im1_sf_positions.append(position)

In [ ]:
#make apertures
aperture  = CircularAperture(im1_sf_positions, r=5)

#show the image with apertures
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
show_image(data, cmap='gray', ax=ax, fig=fig)
aperture.plot(color='green', lw=1.5, alpha=0.5) # apertures
#annulus_aperture.plot(color='red', lw=1.5, alpha=0.5)                        # bg annulus
plt.title('V-NGC6823-1')
plt.tight_layout()
plt.show()

In [ ]:
#the starfinder positions we found in the first image
#are used as the approximate positions for the 2nd image.

image2 = n_infilter[1] #get sources from second image

#the stars are a little bit larger/brighter, so I increased the fwhm and SNR
fwhm       = 5.0
source_snr = 20

sci        = CCDData.read(file,unit='adu')
data       = sci.data[739:1438,880:1660]
hdr        = sci.header

mean, median, std = sigma_clipped_stats(data, sigma=3.0)    
daofind = DAOStarFinder(fwhm=fwhm, threshold=source_snr*std)  
sources_2 = daofind(data) #2nd image starfinder sources

#make array of 2nd image source positions
im2_sf_positions = []

for i in range(len(sources_2['xcentroid'])):
    position = tuple([sources_2['xcentroid'][i],sources_2['ycentroid'][i]])
    im2_sf_positions.append(position)

#make apertures
aperture  = CircularAperture(im1_sf_positions, r=5)
phot_im1 = aperture_photometry(data, aperture)
annulus_aperture = CircularAnnulus(starfinder_pos, r_in=15., r_out=20.)


#show the image with apertures
#apertures are from starfinder running on the 2nd image
#how much do they overlap?
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
show_image(data, cmap='gray', ax=ax, fig=fig)
aperture.plot(color='green', lw=1.5, alpha=0.5) # apertures
#annulus_aperture.plot(color='red', lw=1.5, alpha=0.5)                        # bg annulus
plt.title('V-NGC6823-2')
plt.tight_layout()
plt.show()

#we see on the second image that the first set of apertures fit well over the second image's stars
#so they'll work as apertures to do photometry.

In [ ]:
#plot the position matches on the second image:
#make apertures
aperture  = CircularAperture(im2_sf_positions, r=5)
phot_im2 = aperture_photometry(data, aperture)
#annulus_aperture = CircularAnnulus(starfinder_pos, r_in=15., r_out=20.)

In [ ]:
lan_mag3 = [6]*79

#here's the modified version of the function that handles NGC6823
def get_phota(file,filtr):
    '''
    This function takes an input file, list of positions for stars to find, and plot title.
    
    It executes the following steps:
    1. Read in the data.
    2. Find all sources in the image. It assumes the FWHM is approximately 4 pix and finds everything greater than 10 sigma.
    3. It collects the info for the sources given in the input position list, matching within 10 pixel positions.
    4. It puts r=10pix apertures on each of these stars, as well as r_in=15, r_out=20 background annuli.
    5. It calculates background-subtracted instrumental magnitudes.
    6. It plots the image and apertures for visual inspection.
    7. It returns a table of photometry.
    '''
    fwhm       = 5.0
    source_snr = 20

    sci        = CCDData.read(file,unit='adu')
    data       = sci.data[739:1438,880:1660]
    hdr        = sci.header
    
    mean, median, std = sigma_clipped_stats(data, sigma=3.0)    
    daofind = DAOStarFinder(fwhm=fwhm, threshold=source_snr*std)    
    sources = daofind(data)
    
    #print(sources)
    
    # photometry
    # biggest FWHM is 4.2 pix
    aperture  = CircularAperture(im1_sf_positions, r=5)
    phota = aperture_photometry(data, aperture)
    for col in phota.colnames:
        phota[col].info.format = '%.8g'              # for consistent table output

        
    # calculate the mean background
    # note that for PG1633+099, this will include two little stars :(
    annulus_aperture = CircularAnnulus(im1_sf_positions, r_in=15., r_out=20.)
    bg   = aperture_photometry(data,annulus_aperture)
    
    for col in bg.colnames:
        bg[col].info.format = '%.8g'                # for consistent table output
    msky = bg['aperture_sum']/annulus_aperture.area # get bg cts/pixel
    
    
    phota['inst_mag'] = -2.5 * np.log10(phota['aperture_sum']-msky*aperture.area)
    
    phota['diff_mag'] =phota['inst_mag']-lan_mag3
    

    
    # show the image with apertures
    #fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    #show_image(data, cmap='gray', ax=ax, fig=fig)
    #aperture.plot(color='green', lw=1.5, alpha=0.5) # apertures
    #annulus_aperture.plot(color='red', lw=1.5, alpha=0.5)                        # bg annulus
    #ax.set_xlim([800,1500])
    #ax.set_ylim([800,1500])
    #plt.title(title)
    #plt.tight_layout()
    #plt.show()
    
    return phota

In [ ]:
get_phota(reduced_dir+'a157otzf.fits','V')

In [ ]:
photv = [get_phota(reduced_dir+'a158otzf.fits','V')]
photb = [get_phota(reduced_dir+'a160otzf.fits','B')]

In [ ]:
photv[0][5][:]
photb[0][5][:]
BellBivdeVoe = (photb[0][4][:]) - (photv[0][4][:])

In [ ]:
plt.scatter(BellBivdeVoe,(photv[0][4][:]))
plt.title('Color vs. Magnitude diagram')
plt.show()

In [ ]:
m_o5 = (np.array(photv[0][4][:])) - (k*1.10)
m_o6 = (np.array(photb[0][4][:])) - (k*1.10)
red = k*3.2

BellBivdeVoeCorrected = (m_o6) - (m_o5)
plt.scatter(BellBivdeVoe,(m_naut5))
plt.show()

Our color-mag diagram could be fit with a line, though we don't distinguish between the different types of stars in our image. There seems to be more scatter in the published color-magnitude diagram as opposed to our diagram, most of the stars seem to follow the same curve. Our background subtraction method will also not be as accurate since several of the stars are closely clustered together, and the light from those stars would have been included in the background sky magnitude. Even with our extinction corrections for the atmosphere, there are still reddening effects due to the geometry of interstellar dust that have not been accounted for.